In [6]:
#importing libraries
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
pd.options.display.max_columns = None
import plotly.express as px
from dash import Input, Output
from dash.exceptions import PreventUpdate

In [7]:
#reading dataset
df = pd.read_csv("EmissionData.csv")

In [8]:
#defining emission classes for conditions
emission_classes = {
    "<2.21": lambda x: x < 2.21,
    "2.21 - 5.10": lambda x: 2.21 < x < 5.10,
    "5.10 - 9.64": lambda x: 5.10 < x < 9.64,
    "9.64 - 16.64": lambda x: 9.64 < x < 16.64,
    ">16.64": lambda x: x > 16.64
}

#filling exceptional values with None
df['Emission Class'] = None

for i in range(len(df)):
    emission = df.loc[i, 'Emission']
    for emission_class, condition in emission_classes.items():
        if condition(emission):
            df.loc[i, 'Emission Class'] = emission_class
            break

In [9]:
app = JupyterDash(__name__)

#callback exceptions to be suppressed
app.config.suppress_callback_exceptions=True

#define layout of the app
app.layout = html.Div([
    html.Br(),
    html.H1('Carbon Index - World Bank Data', style={'textAlign': 'center'}),
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.H2("By Country"),
            dcc.Dropdown(
                id='country_dropdown',
                multi=True,
                options=[{'label': country, 'value': country} for country in df['Country'].unique()]
            ),
            html.Br(),
            html.Div(id='linechart-container', children=[
                dcc.Graph(id='country_linechart'),
            ]),
            dcc.RangeSlider(
                id='year_slider',
                marks=None,
                tooltip={"placement": "bottom", "always_visible": True},
                step=1,
                min=df['Year'].min(),
                max=df['Year'].max(),
                value=[df['Year'].min(), df['Year'].max()],
                allowCross=False
            )
        ], style={'width': '48%', 'display': 'inline-block', 'margin-right': '5px'}),
        dbc.Col([
            html.H2("By Year"),
            dcc.Dropdown(
                id='year_dropdown',
                options=[{'label': year, 'value': year} for year in df['Year'].unique()]
            ),
            html.Br(),
            dcc.Tabs(id='tabs', value='tab-1', children=[
                dcc.Tab(label='Bar', value='tab-1'),
                dcc.Tab(label='Map', value='tab-2'),
            ]),
            html.Div(id='tab-content')
        ], style={'width': '48%', 'display': 'inline-block', 'margin-left': '5px'}),
    ]),
])


#callback to update the line chart depending on selected countries and selected year range
@app.callback(
    Output('country_linechart', 'figure'),
    Input('country_dropdown', 'value'),
    Input('year_slider', 'value')
)
def update_linechart(countries, year_range):
    if not countries or not year_range:
        raise PreventUpdate
        
    #linechart year range selection
    line_data = df[(df['Country'].isin(countries)) & (df['Year'].between(year_range[0], year_range[1]))].dropna(
        subset=["Emission"])
    
    #setting figure composition
    line_fig = px.line(
        line_data,
        x='Year',
        y="Emission",
        color='Country',
        width=900,
        title="".join(['<br>', ' - '.join(countries), '</b>']) + " - Carbon Emission"
    )
    #setting layout of legend
    line_fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5))

    line_fig.update_layout(
        title={
            'y': 0.95,
            'x': 0.05,
            'xanchor': 'left',
            'yanchor': 'top'})

    return line_fig

#callback to update the bar chart selected countries on line chart and selected year
@app.callback(
    Output('country_barchart', 'figure'),
    Input('country_dropdown', 'value'),
    Input('year_dropdown', 'value')
)
def update_barchart(countries, selected_year):
    if not countries or not selected_year:
        raise PreventUpdate
        
    #creating bar chart for previously selected countries and selected year
    bar_data = df[(df['Country'].isin(countries)) & (df['Year'] == selected_year)].dropna(subset=["Emission"])
    bar_fig = px.bar(
        bar_data,
        x='Country',
        y="Emission",
        color='Country',
        width=900,
        title="".join(['<br>', ' - '.join(countries), '</b>']) + " - Carbon Emission - Year " + str(selected_year)
    )

    return bar_fig

#callback for the tab content depending on tab value
@app.callback(Output('tab-content', 'children'),
              Input('tabs', 'value'))
def render_tab_content(tab):
    if not tab:
        raise PreventUpdate
    #for tab 1 shows bar chart    
    elif tab == 'tab-1':
        return html.Div([
            html.Div(id='barchart-container', children=[
                dcc.Graph(id='country_barchart'),
            ]),
        ])
    #for tab 2 show choropleth map for selected year
    elif tab == 'tab-2':
        return html.Div([
            html.Div(id='map-container', children=[
                dcc.Graph(id='choropleth_map'),
            ])
        ])
    
#callback to update the choropleth map based on selected year
@app.callback(
    Output('choropleth_map', 'figure'),
    Input('year_dropdown', 'value')
)
def update_choropleth_map(selected_year):
    if not selected_year:
        raise PreventUpdate
        
    #define emission classes and select year
    emission_classes = ["<2.21", "2.21 - 5.10", "5.10 - 9.64", "9.64 - 16.64", ">16.64"]
    map_data = df[df['Year'] == selected_year]
    
    #set choropleth color and title
    fig = px.choropleth(
        map_data,
        locations="Country Code",
        color="Emission Class",
        hover_name="Country Code",
        color_discrete_sequence=px.colors.sequential.Blues,
        title="Carbon Emission in Year " + str(selected_year),
        category_orders={"Emission Class": emission_classes}
    )
    
    #set legend layout of map
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5))

    fig.update_layout(
        title={
            'y': 0.95,
            'x': 0.05,
            'xanchor': 'left',
            'yanchor': 'top'})

    return fig

In [10]:
app.run_server(mode='external', port=1359)

Dash is running on http://127.0.0.1:1359/

Dash app running on http://127.0.0.1:1359/
